In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Battery Data/battery.csv')

<ipython-input-2-5c788b239427>:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('/content/drive/MyDrive/Battery Data/battery.csv')


In [ ]:
property_groups = {prop: df for prop, df in data.groupby('Property')}

In [ ]:
shared_key=[]
shared_pop=[]
for key, df in property_groups.items():
    shared_key.append(key)
    shared_pop.append(df.shape[0])


In [ ]:
#
# Group by 'Property' and collect sets of 'Name' values
from collections import defaultdict

property_name_map = defaultdict(set)

for prop, group_df in data.groupby('Property'):
    property_name_map[prop] = set(group_df['Name'])

# Compare sets to find overlaps
from itertools import combinations

shared_names = {}
shared_name_dfs = {}


for (prop1, names1), (prop2, names2) in combinations(property_name_map.items(), 2):
    overlap = names1 & names2
    if overlap:
        shared_names[(prop1, prop2)] = overlap

# Display shared names between property groups
for (prop1, prop2), names in shared_names.items():
  filtered_df = data[
    (data['Property'].isin([prop1, prop2])) &
        (data['Name'].isin(names))
    ]
  shared_name_dfs[(prop1, prop2)] = filtered_df


In [ ]:
df = pd.DataFrame(data)

# Group by 'Extracted_name'
g = df.groupby('Extracted_name')

# Create a dictionary to store DataFrames for each Extracted_name with different Property values combined
grouped_dfs = {}

# Loop through each group and combine their rows
for name, group in g:
    # Append all the rows with the same Extracted_name into a single DataFrame
    group = group.drop_duplicates(subset='Property', keep='first')
    grouped_dfs[name] = group  # Store the whole group (with all Property values)



In [ ]:
from itertools import combinations
all_data = pd.concat(grouped_dfs.values())

# Get all unique property combinations (pairwise)
unique_properties = all_data['Property'].unique()
property_pairs = combinations(unique_properties, 2)

# Dictionary to store the final dataframes
final_pairwise_dfs = {}

for prop1, prop2 in property_pairs:
    # Filter rows where Property is either prop1 or prop2
    filtered = all_data[all_data['Property'].isin([prop1, prop2])]

    # Group by Extracted_name and keep only those with both properties
    grouped = filtered.groupby('Extracted_name')
    valid_entries = []

    for name, group in grouped:
        if set([prop1, prop2]).issubset(set(group['Property'])):
            # Extract only the two rows and pivot to a single row
            pivoted = group.set_index('Property')['Value'].loc[[prop1, prop2]]
            pivoted_df = pd.DataFrame(pivoted).T
            pivoted_df['Extracted_name'] = name
            valid_entries.append(pivoted_df)

    # Combine all valid entries into one DataFrame
    if valid_entries:
        final_df = pd.concat(valid_entries, ignore_index=True)
        # Reorder columns to have name first
        cols = ['Extracted_name', prop1, prop2]
        final_pairwise_dfs[f"{prop1}-{prop2}"] = final_df[cols]


In [ ]:
final_pairwise_dfs['Capacity-Conductivity'].shape[0]
key_name=[]
key_pop=[]
for key in final_pairwise_dfs.keys():
  key_name.append(str(key))
  key_pop.append(int(final_pairwise_dfs[key].shape[0]))

In [ ]:
!pip install jarvis-tools
import jarvis

In [ ]:
from jarvis.db.figshare import data
d=data('dft_3d')
didf=pd.DataFrame(d)[['jid','formula']]
g=data('cfid_3d')
oqmd = pd.DataFrame(g)[['jid', 'formula']]

Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.
Obtaining 3D dataset 55k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.


In [ ]:
h=data('dft_2d_2021')
dft2_ = pd.DataFrame(h)[['jid', 'formula']]

Obtaining 2D dataset 1.1k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Loading the zipfile...
Loading completed.


In [ ]:
j=data('dft_2d')
dft2 = pd.DataFrame(j)[['jid', 'formula']]

Obtaining 2D dataset 1.1k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815705
Loading the zipfile...
Loading completed.


In [ ]:
k=data('dft_3d_2021')
dft3d = pd.DataFrame(k)[['jid', 'formula']]

Obtaining 3D dataset 55k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Loading the zipfile...
Loading completed.


In [ ]:
l=data('jff')
jff = pd.DataFrame(l)[['jid', 'formula']]

Obtaining JARVIS-FF 2k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Loading the zipfile...
Loading completed.


In [ ]:
m=data('qe_tb')
qetb=pd.DataFrame(m)[['jid','formula']]

Obtaining QETB dataset 860k...
Reference:https://arxiv.org/abs/2112.11585
Loading the zipfile...
Loading completed.


In [ ]:
#creating dataframes for all types of data
capacity = property_groups['Capacity']
voltage=property_groups['Voltage']
energy=property_groups['Energy']
columbic=property_groups['Coulombic Efficiency']
conductivity=property_groups['Conductivity']
voltage_capacity=final_pairwise_dfs['Voltage-Capacity']
voltage_conductivity=final_pairwise_dfs['Voltage-Conductivity']
voltage_energy=final_pairwise_dfs['Voltage-Energy']
voltage_columbic=final_pairwise_dfs['Voltage-Coulombic Efficiency']
capacity_conductivity=final_pairwise_dfs['Capacity-Conductivity']
capacity_energy=final_pairwise_dfs['Capacity-Energy']
capacity_columbic=final_pairwise_dfs['Capacity-Coulombic Efficiency']
conductivity_energy=final_pairwise_dfs['Conductivity-Energy']
conductivity_columbic=final_pairwise_dfs['Conductivity-Coulombic Efficiency']
energy_columbic=final_pairwise_dfs['Energy-Coulombic Efficiency']



In [ ]:
import pandas as pd
import re
from fractions import Fraction
import ast

# Pre-index all dataframes
for df in [oqmd, dft2, dft2_, dft3d, jff, qetb, didf]:
    if 'formula' in df.columns:
        df['formula'] = df['formula'].astype(str)
        df.set_index('formula', inplace=True, drop=False)

# Combine all formula→jid mappings into one dictionary
formula_to_jid = {}

for df in [oqmd, dft2, dft2_, dft3d, jff, qetb, didf]:
    if 'formula' in df.columns and 'jid' in df.columns:
        formula_to_jid.update(df[['formula', 'jid']].dropna().set_index('formula')['jid'].to_dict())

def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

def normalize_formula(formula):
    tokens = re.findall(r'([A-Z][a-z]*)(\d*)', formula)
    tokens = sorted([(el, int(cnt) if cnt else 1) for el, cnt in tokens])
    return ''.join([f"{el}{cnt if cnt > 1 else ''}" for el, cnt in tokens])

def generate_empirical_formula_from_dict(element_dict):
    try:
        element_dict = {k: float(v) for k, v in element_dict.items()}
        fractions = [Fraction(v).limit_denominator() for v in element_dict.values()]
        denominators = [f.denominator for f in fractions]

        lcm = 1
        for d in denominators:
            lcm = lcm * d // gcd(lcm, d)

        formula = ''
        for elem, val in element_dict.items():
            num = int(round(val * lcm))
            formula += f"{elem}{num if num > 1 else ''}"

        return normalize_formula(formula)
    except:
        return 'Non-standard'

def get_jid_fast(formula):
    """Fast lookup from pre-computed formula_to_jid dict, includes normalized fallback."""
    if not formula or not isinstance(formula, str):
        return "Invalid input"
    jid = formula_to_jid.get(formula)
    if jid:
        return jid
    normalized = normalize_formula(formula)
    return formula_to_jid.get(normalized, f"No JID found for formula '{formula}'")

def JID_to_csv(df, excel_name):
    formulas = []
    for item in df['Extracted_name']:
        try:
            formula_list = ast.literal_eval(item)
            formula_dict = formula_list[0]  # First dict in list
            formula = generate_empirical_formula_from_dict(formula_dict)
        except Exception as e:
            formula = 'Non-standard'
        formulas.append(formula)

    # Generate dataframe
    formulas_df = pd.DataFrame({'Formula': formulas})
    mask_valid = formulas_df['Formula'] != 'Non-standard'
    formulas_df.loc[mask_valid, 'jid'] = formulas_df.loc[mask_valid, 'Formula'].map(get_jid_fast)

    # Combine with original DataFrame
    df = df.reset_index(drop=True)
    formulas_df = formulas_df.reset_index(drop=True)
    df['Formula'] = formulas_df['Formula']
    df['jid'] = formulas_df['jid']

    # Write to Excel
    df.to_excel(f'/content/drive/MyDrive/pre-processed2/{excel_name}.xlsx', index=False)


In [ ]:
JID_to_csv(voltage,'voltage')

In [ ]:
JID_to_csv(capacity,'capacity')
JID_to_csv(energy,'energy')
JID_to_csv(columbic,'columbic_efficiency')
JID_to_csv(conductivity,'conductivity')
JID_to_csv(voltage_capacity,'voltage_capacity')
JID_to_csv(voltage_conductivity,'voltage_conductivity')
JID_to_csv(voltage_energy,'voltage_energy')
JID_to_csv(voltage_columbic,'voltage_columbic')
JID_to_csv(capacity_conductivity,'capacity_conductivity')
JID_to_csv(capacity_energy,'capacity_energy')
JID_to_csv(capacity_columbic,'capacity_columbic-efficiency')
JID_to_csv(conductivity_energy,'conductivity_energy')
JID_to_csv(conductivity_columbic,'conductivity_columbic')
JID_to_csv(energy_columbic,'energy_columbic-efficiency')

